## Kemal Demirel: 191104091

## Kadir Çağan Eren: 

In [1]:
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from matplotlib import pyplot as plt
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import os
import cv2
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [19]:
def traverse_directory(path, directory):
    dataset = []
    labels = []
    i = 0
    for dirpath, dirnames, filenames in os.walk(path):
        for filename in filenames:
            file_path = os.path.join(dirpath, filename)
            if(file_path.endswith(".jpg")):
                img = cv2.imread(file_path)
                dataset.append(np.array(img))
                tmp_path = dirpath
                if(directory == "train"):
                    label = tmp_path[tmp_path.find("train/")+len("train/"):]    
                elif directory == "test":
                    label = tmp_path[tmp_path.find("test/")+len("test/"):]  
                elif directory == "validation":  
                    label = tmp_path[tmp_path.find("validation/")+len("validation/"):]  
                labels.append(i)
        i = i + 1
    return dataset, labels

In [20]:
X_train, y_train = traverse_directory(os.getcwd()+"/dataset/train", "train")

In [4]:
X_test, y_test = traverse_directory(os.getcwd()+"/dataset/test", "test")

In [21]:
def preprocess_image(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (64, 64))
    return image

In [22]:
for i in range(len(X_test)):
    X_train[i] = preprocess_image(X_train[i])

In [23]:
for i in range(len(X_test)):
    X_test[i] = preprocess_image(X_test[i])

In [24]:
X = np.array(X_train)
y = np.array(y_train)
X_train_flat = X.reshape((X.shape[0], -1))

/var/folders/f_/gyqrcdtx623ff04lttt2nxm40000gn/T/ipykernel_6307/2740003408.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X = np.array(X_train)


In [26]:
# Train a random forest classifier
clf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
clf.fit(X_train_flat, y)

ValueError: setting an array element with a sequence.

In [9]:
# Step 3: Train a model
X_train_flat = np.array([x.flatten() for x in X_train])
clf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
clf.fit(X_train_flat, y_train)

/var/folders/f_/gyqrcdtx623ff04lttt2nxm40000gn/T/ipykernel_6307/2316019646.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_train_flat = np.array([x.flatten() for x in X_train])


ValueError: setting an array element with a sequence.

In [ ]:
# Step 4: Test the model
X_test_flat = np.array([x.flatten() for x in X_test])
y_pred = clf.predict(X_test_flat)
print(classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))

In [ ]:
def feature_extraction_with_sift(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(gray, None)
    max_descriptors = 50  # or any other value that is large enough to accomodate all the descriptors
    if descriptors is not None and descriptors.shape[0] > max_descriptors:
        descriptors = descriptors[:max_descriptors, :]
    elif descriptors is not None and descriptors.shape[0] < max_descriptors:
        descriptors = np.pad(descriptors, ((0, max_descriptors - descriptors.shape[0]), (0, 0)), 'constant')
    if descriptors is None:
        descriptors = np.zeros((max_descriptors, 128))
    return descriptors.flatten()

In [ ]:
def get_features(dataset):
    dataset_features = []
    for image in dataset:
        features = feature_extraction_with_sift(image)
        if features is not None:
            dataset_features.append(features)
    dataset_features = np.array(dataset_features, dtype='object')

In [ ]:
X_train_features = get_features(X_train)

In [ ]:
X_test_features = get_features(X_test)

In [ ]:
X_validation_features = get_features(X_validation)

In [ ]:
import cv2
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import os

# Step 1: Load and preprocess the dataset
DATA_DIR = "fruit_images/"
fruit_images = []
labels = []

for fruit_dir in os.listdir(DATA_DIR):
    if not os.path.isdir(os.path.join(DATA_DIR, fruit_dir)):
        continue

    for filename in os.listdir(os.path.join(DATA_DIR, fruit_dir)):
        if filename.endswith(".jpg"):
            img_path = os.path.join(DATA_DIR, fruit_dir, filename)
            img = cv2.imread(img_path)
            img = cv2.resize(img, (100, 100))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            fruit_images.append(img)
            labels.append(fruit_dir)

# Step 2: Split the dataset into training and testing sets
le = LabelEncoder()
labels = le.fit_transform(labels)
X_train, X_test, y_train, y_test = train_test_split(fruit_images, labels, test_size=0.2, random_state=42)

# Step 3: Train a model
X_train_flat = np.array([x.flatten() for x in X_train])
clf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
clf.fit(X_train_flat, y_train)

# Step 4: Test the model
X_test_flat = np.array([x.flatten() for x in X_test])
y_pred = clf.predict(X_test_flat)
print(classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))

In [ ]:
def train_classifier(X_train, y_train):
    svm = SVC(kernel='linear')
    svm.fit(X_train, y_train)
    return svm

In [ ]:
def evaluate_classifier(classifier, X_test, y_test):
    y_pred = classifier.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='macro')
    recall = recall_score(y_test, y_pred, average='macro')
    f1 = f1_score(y_test, y_pred, average='macro')
    return accuracy, precision, recall, f1

In [ ]:
def predict(classifier, image):
    feature = feature_extraction_with_sift(image)
    prediction = classifier.predict([feature])
    return prediction[0]

In [ ]:
classifier = train_classifier(X_train_features, y_train)

In [ ]:
accuracy, precision, recall, f1 = evaluate_classifier(classifier, X_test_features, y_test)

In [ ]:
print(accuracy)

In [ ]:

new_image = cv2.imread("/Users/kemaldemirel/Desktop/Lectures/Bil 468/Project/Computer-Vision-Fruit-Vegetables-Detection/dataset/validation/cabbage/Image_1.jpg")
new_image = cv2.cvtColor(new_image, cv2.COLOR_BGR2RGB)
new_image = cv2.resize(new_image, (64, 64))
prediction = predict(classifier, new_image)
print(prediction)